In [1]:
%%capture
!pip install name-matching awswrangler ray

In [2]:
import pandas as pd
from name_matching.name_matcher import NameMatcher
import awswrangler as wr
import ray
import numpy as np

In [3]:
patstat_names_query = """SELECT distinct(trim(replace(psn_name, '"', ''))) as firm_name, 
                                trim(replace(person_ctry_code, '"', '')) as country_code
                      from tls206_person 
                      where psn_sector='"COMPANY"'
                      or han_name='"COMPANY"'
                      """
df = wr.athena.read_sql_query(patstat_names_query, database="patstat-global-spring-2021")

In [4]:
df

,firm_name,country_code
0,ZHEJIANG YUELONG LANDSCAPE CONSTRUCTION CO LTD,LTD.
1,FOUR FLAGS,AU
2,SHANXI LONGZHI ENERGY CHEMICAL CO LTD,LTD.
3,BELL FLAVORS & FRAGRANCES,DE
4,YANGZHOU HERUI PREC MANUFACTURE CO LTD,LTD.
...,...,...
4900288,FA AG EISENHUTTE PRINZ R,
4900289,ENSINK NV MASCHINEFAB EN IJZERGIETERIJ,
4900290,BRITISH IRON AND STEEL RES ASS,
4900291,PERET A CO FOR PLANNING AND DEV OF IDEAS FOR I...,


In [5]:
country_counts = df[df.country_code.str.len()==2].country_code.value_counts()
country_counts.head(30)

US    202764
DE    184328
GB     77433
IT     75953
FR     74996
CN     57344
JP     41694
CH     40936
SE     36237
CA     34560
KR     31944
RU     30193
NL     27933
AU     27215
BR     24168
SU     24058
TW     19440
AT     15519
PL     14787
DK     13870
TR     13035
IL     12757
FI     12672
ES     12137
NO     10583
BE      9868
DD      8006
HU      7240
HK      6479
CZ      5526
Name: country_code, dtype: Int64

In [6]:
num_patstat_firms_with_country_code = country_counts.sum()
print(f'Share of Patstat firms with a valid country code: {num_patstat_firms_with_country_code/len(df):.2f}')

Share of Patstat firms with a valid country code: 0.25


In [7]:
orbis_lexis = pd.read_csv('s3://alliance-network/lexis_alliances_orbis_static.csv')
r_and_d = pd.read_csv('s3://alliance-network/ResearchandDevelopment_LexisNexis.csv')

In [8]:
r_and_d_firm_ids = set(r_and_d['firm_a'].unique()) | set(r_and_d['firm_b'].unique())
orbis_lexis = orbis_lexis[orbis_lexis['BvD ID number'].isin(r_and_d_firm_ids)]

In [9]:
orbis_lexis

,Company name Latin alphabet,BvD ID number,Country ISO code,City\nLatin Alphabet,"NACE Rev. 2, core code (4 digits)","NACE Rev. 2, secondary code(s)"
0,AMERICAN UNIVERSITY OF SHARJAH,AE0000025189,AE,SHARJAH,8541.0,8542.0
1,CRESCENT PETROLEUM COMPANY,AE0000037027,AE,SHARJAH,7112.0,NaN
32,ALPHA DATA LLC,AE0000039967,AE,ABU DHABI,4652.0,4778.0
58,ABU DHABI NATIONAL OIL COMPANY (ADNOC),AE0000047504,AE,ABU DHABI,610.0,7112.0
65,JUMBO ELECTRONICS CO. LTD. (L.L.C.),AE0000048486,AE,DUBAI,4669.0,6202.0
...,...,...,...,...,...,...
165905,DELTA CORPORATION LIMITED,ZW30009KZ,ZW,"HARARE, HARARE",1107.0,NaN
165910,TURNALL HOLDINGS LIMITED,ZW30039KZ,ZW,"HARARE, HARARE",3522.0,NaN
165911,CFI HOLDINGS LIMITED,ZW30047KZ,ZW,"HARARE, HARARE",6420.0,NaN
165920,AXIA CORPORATION LIMITED,ZW30092KZ,ZW,HARARE,7022.0,NaN


In [10]:
# ## add country codes to name match column
# df['firm_name_match'] = df.firm_name.str.strip() + ' ' + df.country_code.str.strip()
# orbis_lexis['firm_name_match'] = orbis_lexis['Company name Latin alphabet'].str.strip() + ' ' + orbis_lexis['Country ISO code'].str.strip()

In [11]:
ray.init()

2023-03-24 13:15:33,864	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.8
Ray version:,2.3.0


In [12]:
cpu_count = ray.nodes()[0]['Resources']['CPU']
cpu_count

36.0

In [13]:
def split_df(df, chunk_size):
    num_chunks = len(df) // chunk_size + 1
    return np.array_split(df, num_chunks)

@ray.remote
def match_names(df, column_matching):
    matcher = NameMatcher(low_memory=False, top_n=5, common_words=False, legal_suffixes=True,
                          distance_metrics=['editex', 'discounted_levenshtein',
                                            'refined_soundex'],
                         verbose=True)
    matcher.load_and_process_master_data('Company name Latin alphabet', orbis_lexis)
    res = matcher.match_names(to_be_matched=df, column_matching=column_matching)
    matches = res[res.score > 95]
    return matches

def match_names_multiprocessing(df, column_matching, chunk_size=10000):
    df_batches = split_df(df, chunk_size)
    futures = [match_names.remote(df, column_matching) for df in df_batches]
    matches = ray.get(futures)
    return pd.concat(matches)

In [14]:
matches = match_names_multiprocessing(df, 'firm_name', chunk_size=len(df)//cpu_count+1)

(match_names pid=13449) done


100%|██████████| 136119/136119 [37:24<00:00, 60.64it/s] 


In [16]:
matches.to_pickle('matches.pkl')

In [23]:
matches

,original_name,match_name,score,match_index
105,embrex inc,embrex,100.000000,8657.0
117,volvo technology corp,volvo technology ab,100.000000,6918.0
313,aisin seiki,aisin seiki company limited,100.000000,3538.0
555,ortho clinical diagnostics inc,orthoclinical diagnostics inc,97.188593,11537.0
646,robert bosch,robert bosch srl,100.000000,6378.0
...,...,...,...,...
4899397,tokyo electron ltd,tokyo electron limited,100.000000,5184.0
4899747,top victory investment,top victory investments limited,95.172938,3905.0
4899777,artemis ood,artemis,100.000000,2673.0
4900048,pfenex inc,pfenex inc,100.000000,13892.0


In [21]:
merge_matches = matches.reset_index(names='df_index')[['match_index', 'df_index']]
merged = orbis_lexis.reset_index(drop=True).merge(merge_matches, left_index=True, right_on='match_index')
merged = merged.merge(df, left_on='df_index', right_index=True).drop(columns=['match_index', 'df_index'])

In [34]:
merged

,Company name Latin alphabet,BvD ID number,Country ISO code,City\nLatin Alphabet,"NACE Rev. 2, core code (4 digits)","NACE Rev. 2, secondary code(s)",firm_name,country_code
9077,ALPHA DATA LLC,AE0000039967,AE,ABU DHABI,4652.0,4778.0,ALPHA DATA,
43264,ALPHA DATA LLC,AE0000039967,AE,ABU DHABI,4652.0,4778.0,ALPHA DATA,JP
15767,ABU DHABI NATIONAL OIL COMPANY (ADNOC),AE0000047504,AE,ABU DHABI,610.0,7112.0,ABU DHABI NATIONAL OIL COMPANY (ADNOC),AE
2685,SANY HEAVY INDUSTRY CO. LTD,AE0000588591,AE,DUBAI,4752.0,NaN,SANY HEAVY INDUSTRY COMPANY,
9402,SANY HEAVY INDUSTRY CO. LTD,AE0000588591,AE,DUBAI,4752.0,NaN,サニー ヘビー インダストリー カンパニー リミテッドＳＡＮＹ ＨＥＡＶＹ ＩＮＤＵＳＴＲＹ...,
...,...,...,...,...,...,...,...,...
2555,TRIANGLE LIMITED,ZW0000033705,ZW,TRIANGLE,1081.0,NaN,TRIANGLE,DE
16940,TRIANGLE LIMITED,ZW0000033705,ZW,TRIANGLE,1081.0,NaN,TRIANGLE,
22258,TRIANGLE LIMITED,ZW0000033705,ZW,TRIANGLE,1081.0,NaN,TRIANGLE,FR
29507,TRIANGLE LIMITED,ZW0000033705,ZW,TRIANGLE,1081.0,NaN,TRIANGLE CO LTD,LTD.


In [30]:
merged.to_parquet('patstat-orbis-name-matches.parquet')

In [44]:
name_orbis_correspondence = merged.drop_duplicates('firm_name')[['BvD ID number', 'firm_name']]
name_orbis_correspondence.columns = ['bvdid', 'firm_name']
name_orbis_correspondence.to_parquet('patstat-orbis-name-bvdid-correspondence.parquet')

In [33]:
df.to_parquet('patstat-orbis-originial-df.parquet')